In [ ]:
crash_data = pd.read_csv('../data/clean/crashes.csv')
crash_data.describe()

In [ ]:
crash_data.isna().sum(axis = 0)

In [ ]:
null_loc = crash_data[crash_data.isnull().any(axis=1)]
null_loc

Location data is important for the first step of identifying "hot spots", so any nulls in these columns will need to be addressed. The records with nulls for location fields (`lat`, `long`, & `mapped_location`) do seem to be from actual events and not errors, but given that at the time of this analysis they represent 0.04% of the data, I don't believe removeing them will negatively impact the overall analysis.

In [ ]:
crash_data.dropna()

Need to look at these on a map to find the crash "hot spots"

In [ ]:
county = gpd.read_file('../data/clean/Davidson County Border (GIS).geojson')
# Use ID 0 for centroid
center = county.geometry.centroid[0]
map_center = [center.y, center.x]
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
crash_data_geo = crash_data[['accident_number', 'lat', 'long']]
crashes_geo = gpd.GeoDataFrame(crash_data_geo, crs= zipcodes.crs, geometry=gpd.points_from_xy(crash_data_geo.long, crash_data_geo.lat))

In [ ]:
crashes_geo.dropna()

In [ ]:
crashes_geo.isna().sum(axis = 0)

In [ ]:
# Create the map
all_crashes = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)
folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.5}).add_to(all_crashes)
crashes = []
for row_index, row_values in crashes_geo.iterrows():
    loc = [row_values['lat'], row_values['long']]
    crashes.append(loc)
crashes

HeatMap(crashes, radius = 15, min_opacity = 0.8, gradient={.8: '#ffc2c2', .95: '#ff7970', 1: '#ff0000'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(all_crashes))
folium.LayerControl().add_to(all_crashes)

all_crashes.save('../maps/all_crashes.html')

**For Reference**

Syntax for SQL queries in Python:
```
query_name = '''
(
SQL syntax
)
'''

with engine.connect() as connection:
    counts = pd.read_sql(text(query_name), con = connection)
```